### Import libraries

In [422]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import Data

In [423]:
population = pd.read_csv("./data/country_population.csv")
fertility = pd.read_csv("./data/fertility_rate.csv")
life_expectancy = pd.read_csv("./data/life_expectancy.csv")

### Treat data

In [424]:
unwanted_cols = ["Indicator Name", "Indicator Code", "Country Name"]

population.drop(unwanted_cols, axis=1, inplace=True)
fertility.drop(unwanted_cols, axis=1, inplace=True)
life_expectancy.drop(unwanted_cols, axis=1, inplace=True)

population = population.transpose()
fertility = fertility.transpose()
life_expectancy = life_expectancy.transpose()

years = pd.DataFrame(list(population.index)[1:])

population = pd.concat([years, pd.DataFrame(population.values[1:], columns = population.iloc[0])], axis=1).rename(columns= {0: "year"})
fertility = pd.concat([years, pd.DataFrame(fertility.values[1:], columns = fertility.iloc[0])], axis=1).rename(columns= {0: "year"})
life_expectancy = pd.concat([years, pd.DataFrame(life_expectancy.values[1:], columns = life_expectancy.iloc[0])], axis=1).rename(columns= {0: "year"})

### Transform into deltas

In [425]:
dsts = [population, fertility, life_expectancy]
dst_names = ["population", "fertility", "life_expectancy"]

population_deltas = []
fertility_deltas = []
life_expectancy_deltas = []

for i in range(3):
    dst = dsts[i]
    for col in dst.columns:
        delta_col = []
        if col != "year":
            for j in range(len(dst[col]) - 1):
                num = dst[col][j]
                delta = dst[col][j + 1] - num
                delta_col.append(delta)
            if(i == 0):
                population_deltas.append(delta_col)
            if(i == 1):
                fertility_deltas.append(delta_col)
            if(i == 2):
                life_expectancy_deltas.append(delta_col)

### Split deltas into X and Y

In [426]:
import math

def splitDeltas(k, deltas):
    X = []
    y = []

    for arr in deltas:
        temp_X = []
        for d in arr:
            if(math.isnan(d)):
                temp_X = []
                continue
            if(len(temp_X) == k):
                X.append(temp_X)
                y.append(d)
                temp_X = []

            temp_X.append(d)
    return X, y
        

X_population_k3, y_population_k3  = splitDeltas(3, population_deltas)
X_fertility_k3, y_fertility_k3 = splitDeltas(3, fertility_deltas)
X_life_expectancy_k3, y_life_expectancy_k3 = splitDeltas(3, life_expectancy_deltas)

X_population_k6, y_population_k6  = splitDeltas(6, population_deltas)
X_fertility_k6, y_fertility_k6 = splitDeltas(6, fertility_deltas)
X_life_expectancy_k6, y_life_expectancy_k6 = splitDeltas(6, life_expectancy_deltas)

X_population_k10, y_population_k10  = splitDeltas(10, population_deltas)
X_fertility_k10, y_fertility_k10 = splitDeltas(10, fertility_deltas)
X_life_expectancy_k10, y_life_expectancy_k10 = splitDeltas(10, life_expectancy_deltas)

# Models

### Regression Metrics Function

In [427]:
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error, matthews_corrcoef, max_error, accuracy_score
from scipy.stats import pearsonr

def printRegStatistics(truth, preds):
    corr, pval = pearsonr(truth, preds)
    print("The RVE is: %7.4f" %explained_variance_score(truth, preds))
    print("The RMSE is: %7.4f" %mean_squared_error(truth, preds, squared=False))
    print("The Correlation Score is: %6.4f (p-value=%e)"%(corr,pval))
    print("The Maximum Error is: %7.4f" %max_error(truth, preds))
    print("The Mean Absolute Error: %7.4f" %mean_absolute_error(truth, preds))
    print("------------------------------")

### Split the data

In [428]:
from sklearn.model_selection import train_test_split

# K3
X_train_IVS_k3_population, X_IVS_k3_population, y_train_IVS_k3_population, y_IVS_k3_population = train_test_split(X_population_k3, y_population_k3, test_size=0.20, random_state=7)
X_train_k3_population, X_test_k3_population, y_train_k3_population, y_test_k3_population = train_test_split(X_train_IVS_k3_population, y_train_IVS_k3_population, test_size=0.20, random_state=7)

X_train_IVS_k3_fertility, X_IVS_k3_fertility, y_train_IVS_k3_fertility, y_IVS_k3_fertility = train_test_split(X_fertility_k3, y_fertility_k3, test_size=0.20, random_state=7)
X_train_k3_fertility, X_test_k3_fertility, y_train_k3_fertility, y_test_k3_fertility = train_test_split(X_train_IVS_k3_fertility, y_train_IVS_k3_fertility, test_size=0.20, random_state=7)

X_train_IVS_k3_life_expectancy, X_IVS_k3_life_expectancy, y_train_IVS_k3_life_expectancy, y_IVS_k3_life_expectancy = train_test_split(X_life_expectancy_k3, y_life_expectancy_k3, test_size=0.20, random_state=7)
X_train_k3_life_expectancy, X_test_k3_life_expectancy, y_train_k3_life_expectancy, y_test_k3_life_expectancy = train_test_split(X_train_IVS_k3_life_expectancy, y_train_IVS_k3_life_expectancy, test_size=0.20, random_state=7)

# K6
X_train_IVS_k6_population, X_IVS_k6_population, y_train_IVS_k6_population, y_IVS_k6_population = train_test_split(X_population_k6, y_population_k6, test_size=0.20, random_state=7)
X_train_k6_population, X_test_k6_population, y_train_k6_population, y_test_k6_population = train_test_split(X_train_IVS_k6_population, y_train_IVS_k6_population, test_size=0.20, random_state=7)

X_train_IVS_k6_fertility, X_IVS_k6_fertility, y_train_IVS_k6_fertility, y_IVS_k6_fertility = train_test_split(X_fertility_k6, y_fertility_k6, test_size=0.20, random_state=7)
X_train_k6_fertility, X_test_k6_fertility, y_train_k6_fertility, y_test_k6_fertility = train_test_split(X_train_IVS_k6_fertility, y_train_IVS_k6_fertility, test_size=0.20, random_state=7)

X_train_IVS_k6_life_expectancy, X_IVS_k6_life_expectancy, y_train_IVS_k6_life_expectancy, y_IVS_k6_life_expectancy = train_test_split(X_life_expectancy_k6, y_life_expectancy_k6, test_size=0.20, random_state=7)
X_train_k6_life_expectancy, X_test_k6_life_expectancy, y_train_k6_life_expectancy, y_test_k6_life_expectancy = train_test_split(X_train_IVS_k6_life_expectancy, y_train_IVS_k6_life_expectancy, test_size=0.20, random_state=7)

# K10
X_train_IVS_k10_population, X_IVS_k10_population, y_train_IVS_k10_population, y_IVS_k10_population = train_test_split(X_population_k10, y_population_k10, test_size=0.20, random_state=7)
X_train_k10_population, X_test_k10_population, y_train_k10_population, y_test_k10_population = train_test_split(X_train_IVS_k10_population, y_train_IVS_k10_population, test_size=0.20, random_state=7)

X_train_IVS_k10_fertility, X_IVS_k10_fertility, y_train_IVS_k10_fertility, y_IVS_k10_fertility = train_test_split(X_fertility_k10, y_fertility_k10, test_size=0.20, random_state=7)
X_train_k10_fertility, X_test_k10_fertility, y_train_k10_fertility, y_test_k10_fertility = train_test_split(X_train_IVS_k10_fertility, y_train_IVS_k10_fertility, test_size=0.20, random_state=7)

X_train_IVS_k10_life_expectancy, X_IVS_k10_life_expectancy, y_train_IVS_k10_life_expectancy, y_IVS_k10_life_expectancy = train_test_split(X_life_expectancy_k10, y_life_expectancy_k10, test_size=0.20, random_state=7)
X_train_k10_life_expectancy, X_test_k10_life_expectancy, y_train_k10_life_expectancy, y_test_k10_life_expectancy = train_test_split(X_train_IVS_k10_life_expectancy, y_train_IVS_k10_life_expectancy, test_size=0.20, random_state=7)

### Variables for Later

In [429]:
import sys

bestOverallRMSE_population = sys.float_info.max
bestOverallRMSEParams_population = {
    "model": "",
    "k": ""
}

bestOverallRMSE_fertility = sys.float_info.max
bestOverallRMSEParams_fertility = {
    "model": "",
    "k": ""
}

bestOverallRMSE_life_expectancy = sys.float_info.max
bestOverallRMSEParams_life_expectancy = {
    "model": "",
    "k": ""
}

### Model 1 - Linear Regression

In [430]:
from sklearn.linear_model import LinearRegression

for dst in ["population", "fertility", "life_expectancy"]:
    bestRMSE_LR = sys.float_info.max
    bestK_LR = ""

    for k in ["3", "6", "10"]:
        X_train = globals()["X_train_k" + k + "_" + dst]
        y_train = globals()["y_train_k" + k + "_" + dst]
        X_test = globals()["X_test_k" + k + "_" + dst]
        y_test = globals()["y_test_k" + k + "_" + dst]

        lr = LinearRegression()
        lr.fit(X_train, y_train)

        preds = lr.predict(X_test)
        
        rmse = mean_squared_error(y_test, preds, squared=False)

        # print("LR")
        # print(dst, k)
        # print(printRegStatistics(y_test, preds))

        if(bestRMSE_LR > rmse):
            bestRMSE_LR = rmse
            bestK_LR = k
    
    print("LR |", dst, "| K =", bestK_LR, "| RMSE =", bestRMSE_LR)

    if(bestRMSE_LR < globals()["bestOverallRMSE_" + dst]):
        globals()["bestOverallRMSE_" + dst] = bestRMSE_LR
        globals()["bestOverallRMSEParams_" + dst]["model"] = "Linear Regression"
        globals()["bestOverallRMSEParams_" + dst]["k"] = bestK_LR

LR | population | K = 10 | RMSE = 253641.98523878766
LR | fertility | K = 10 | RMSE = 0.060746290934078816
LR | life_expectancy | K = 10 | RMSE = 0.19469746568244908


### Model 2 - Random Forest

In [431]:
from sklearn.ensemble import RandomForestRegressor

for dst in ["population", "fertility", "life_expectancy"]:
    bestRMSE_RF = sys.float_info.max
    bestK_RF = ""
    best_RF_params = {}

    for k in ["3", "6", "10"]:
        X_train = globals()["X_train_k" + k + "_" + dst]
        y_train = globals()["y_train_k" + k + "_" + dst]
        X_test = globals()["X_test_k" + k + "_" + dst]
        y_test = globals()["y_test_k" + k + "_" + dst]

        bestRMSE = sys.float_info.max
        bestRMSEParams = {
            "max_depth": 0,
            "min_samples_leaf": 0 
        }

        for depth in range(2, 11):
            for leaf in range(25, 150, 25):
                rf = RandomForestRegressor(n_estimators = 50, random_state = 7, max_depth = depth, min_samples_leaf = leaf)
                rf.fit(X_train, y_train)
                preds = rf.predict(X_test)

                rmse = mean_squared_error(y_test, preds, squared=False)

                if(bestRMSE > rmse):
                    bestRMSE = rmse
                    bestRMSEParams["max_depth"] = depth
                    bestRMSEParams["min_samples_leaf"] = leaf
        
        if(bestRMSE < bestRMSE_RF):
            bestRMSE_RF = bestRMSE
            bestK_RF = k
            best_RF_params.update(bestRMSEParams) 

    print("RF |", dst, "| K =", bestK_RF, "| RMSE =", bestRMSE_RF , "| With:", best_RF_params)
    
    # rf = RandomForestRegressor(n_estimators = 50, random_state = 7, max_depth = best_RF_params['max_depth'], min_samples_leaf = best_RF_params['min_samples_leaf'])
    # rf.fit(globals()["X_train_k" + bestK_RF + "_" + dst], globals()["y_train_k" + bestK_RF + "_" + dst])
    # preds = rf.predict(globals()["X_test_k" + bestK_RF + "_" + dst])
    # print(printRegStatistics(globals()["y_test_k" + bestK_RF + "_" + dst], preds))

    if(bestRMSE_RF < globals()["bestOverallRMSE_" + dst]):
        globals()["bestOverallRMSE_" + dst] = bestRMSE_RF
        globals()["bestOverallRMSEParams_" + dst]["model"] = "Random Forest"
        globals()["bestOverallRMSEParams_" + dst]["k"] = bestK_RF
        globals()["bestOverallRMSEParams_" + dst].update(best_RF_params)    

RF | population | K = 3 | RMSE = 1596955.984387569 | With: {'max_depth': 9, 'min_samples_leaf': 25}
RF | fertility | K = 10 | RMSE = 0.05425216457617322 | With: {'max_depth': 2, 'min_samples_leaf': 75}
RF | life_expectancy | K = 6 | RMSE = 0.18025024965000402 | With: {'max_depth': 5, 'min_samples_leaf': 50}


### The best models are...

In [432]:
for dst in ["population", "fertility", "life_expectancy"]:
    print("The best model for", dst, "is:")
    print(globals()["bestOverallRMSEParams_" + dst])
    print("-------")

The best model for population is:
{'model': 'Linear Regression', 'k': '10'}
-------
The best model for fertility is:
{'model': 'Random Forest', 'k': '10', 'max_depth': 2, 'min_samples_leaf': 75}
-------
The best model for life_expectancy is:
{'model': 'Random Forest', 'k': '6', 'max_depth': 5, 'min_samples_leaf': 50}
-------


## Fit and validate best models

### Population

In [433]:
lr_population = LinearRegression()
lr_population.fit(X_train_IVS_k10_population, y_train_IVS_k10_population)

preds = lr_population.predict(X_IVS_k10_population)
printRegStatistics(y_IVS_k10_population, preds)

The RVE is:  0.9996
The RMSE is: 212868.2143
The Correlation Score is: 0.9998 (p-value=0.000000e+00)
The Maximum Error is: 1673025.8782
The Mean Absolute Error: 68749.7940
------------------------------


### Fertility

In [434]:
rf_fertility = RandomForestRegressor(n_estimators = 50, random_state = 7, max_depth = 2, min_samples_leaf = 75)
rf_fertility.fit(X_train_IVS_k10_fertility, y_train_IVS_k10_fertility)

preds = rf_fertility.predict(X_IVS_k10_fertility)
printRegStatistics(y_IVS_k10_fertility, preds)

The RVE is:  0.5201
The RMSE is:  0.0472
The Correlation Score is: 0.7225 (p-value=5.077343e-40)
The Maximum Error is:  0.2305
The Mean Absolute Error:  0.0272
------------------------------


### Life Expectancy

In [435]:
rf_life_expectancy = RandomForestRegressor(n_estimators = 50, random_state = 7, max_depth = 5, min_samples_leaf = 50)
rf_life_expectancy.fit(X_train_IVS_k6_life_expectancy, y_train_IVS_k6_life_expectancy)

preds = rf_life_expectancy.predict(X_IVS_k6_life_expectancy)
printRegStatistics(y_IVS_k6_life_expectancy, preds)

The RVE is:  0.4437
The RMSE is:  0.2931
The Correlation Score is: 0.6663 (p-value=9.177680e-57)
The Maximum Error is:  3.1109
The Mean Absolute Error:  0.1234
------------------------------


## Select 10 random countries

In [436]:
import random

country_codes = list(population.columns[1:])
country_codes_filtered = []

for col in population:
    if col != "year":
        country = col
        if not np.isnan(list(population[col])[-10:]).any() and not np.isnan(list(fertility[col])[-10:]).any() and not np.isnan(list(life_expectancy[col])[-10:]).any():
            country_codes_filtered.append(country)

random.seed(120)

selected_countries = []

for i in range(0, 10):
    n = random.randint(0, len(country_codes_filtered) - 1)
    selected_countries.append(country_codes_filtered[n])

print(selected_countries)

selected_countries_index = []

for country in selected_countries:
    selected_countries_index.append(country_codes.index(country))

print(selected_countries_index)

['LUX', 'EST', 'CZE', 'ZMB', 'USA', 'NPL', 'SAU', 'MRT', 'PNG', 'ZAF']
[142, 69, 52, 262, 249, 176, 203, 164, 187, 261]


## Population | 2017

In [447]:
preds_population = []

for i in selected_countries_index:
    last_10_deltas = population_deltas[i][-10:]
    delta = int(lr_population.predict([last_10_deltas])[0])
    value_2016 = int(list(population[country_codes[i]])[-1])
    pred_2017 = int(value_2016 + delta)
    preds_population.append(pred_2017)
    print("For country", country_codes[i], "|", "Delta 2016-2017:", delta, "|", "Value 2016:", value_2016, "|", "Predicted Value 2017:", pred_2017)
    print("---------------------")

For country LUX | Delta 2016-2017: 8175 | Value 2016: 582014 | Predicted Value 2017: 590189
---------------------
For country EST | Delta 2016-2017: -695 | Value 2016: 1315790 | Predicted Value 2017: 1315095
---------------------
For country CZE | Delta 2016-2017: 43991 | Value 2016: 10566332 | Predicted Value 2017: 10610323
---------------------
For country ZMB | Delta 2016-2017: 486001 | Value 2016: 16591390 | Predicted Value 2017: 17077391
---------------------
For country USA | Delta 2016-2017: 2309742 | Value 2016: 323127513 | Predicted Value 2017: 325437255
---------------------
For country NPL | Delta 2016-2017: 307500 | Value 2016: 28982771 | Predicted Value 2017: 29290271
---------------------
For country SAU | Delta 2016-2017: 643722 | Value 2016: 32275687 | Predicted Value 2017: 32919409
---------------------
For country MRT | Delta 2016-2017: 111382 | Value 2016: 4301018 | Predicted Value 2017: 4412400
---------------------
For country PNG | Delta 2016-2017: 159657 | Value 

### Truth / Predicted Plot | Population

In [438]:
population_2017 = list(pd.read_csv("./data/population2017.csv", header=None)[1])

def plotGraph(preds, truth, color, title):
    plt.figure(figsize=(7, 7))
    plt.title(title)
    plt.scatter(preds, truth, c=color)
    plt.grid()
    plt.axline([0, 0], [1, 1], c="black")
    plt.show()

# plotGraph(preds_population, population_2017, "blue", "Population")

## Fertility | 2017

In [451]:
preds_fertility = []

for i in selected_countries_index:
    last_10_deltas = fertility_deltas[i][-10:]
    delta = round(rf_fertility.predict([last_10_deltas])[0], 2)
    value_2016 = list(fertility[country_codes[i]])[-1]
    pred_2017 = round(value_2016 + delta, 2)
    preds_fertility.append(pred_2017)
    print("For country", country_codes[i], "|", "Delta 2016-2017:", delta, "|", "Value 2016:", value_2016, "|", "Predicted Value 2017:", pred_2017)
    print("---------------------")

For country LUX | Delta 2016-2017: -0.02 | Value 2016: 1.47 | Predicted Value 2017: 1.45
---------------------
For country EST | Delta 2016-2017: -0.01 | Value 2016: 1.58 | Predicted Value 2017: 1.57
---------------------
For country CZE | Delta 2016-2017: -0.01 | Value 2016: 1.57 | Predicted Value 2017: 1.56
---------------------
For country ZMB | Delta 2016-2017: -0.05 | Value 2016: 4.981 | Predicted Value 2017: 4.93
---------------------
For country USA | Delta 2016-2017: -0.04 | Value 2016: 1.8 | Predicted Value 2017: 1.76
---------------------
For country NPL | Delta 2016-2017: -0.05 | Value 2016: 2.118 | Predicted Value 2017: 2.07
---------------------
For country SAU | Delta 2016-2017: -0.05 | Value 2016: 2.532 | Predicted Value 2017: 2.48
---------------------
For country MRT | Delta 2016-2017: -0.05 | Value 2016: 4.674 | Predicted Value 2017: 4.62
---------------------
For country PNG | Delta 2016-2017: -0.05 | Value 2016: 3.658 | Predicted Value 2017: 3.61
-------------------

### Truth / Predicted Plot | Fertility

In [440]:
fertility_2017 = list(pd.read_csv("./data/fertility2017.csv", header=None)[1])
# plotGraph(preds_fertility, fertility_2017, "magenta", "Fertility")

## Life Expectancy | 2017

In [453]:
preds_life_expect = []

for i in selected_countries_index:
    last_6_deltas = life_expectancy_deltas[i][-6:]
    delta = round(rf_life_expectancy.predict([last_6_deltas])[0], 2)
    value_2016 = round(list(life_expectancy[country_codes[i]])[-1], 2)
    pred_2017 = round(value_2016 + delta, 2)
    preds_life_expect.append(pred_2017)
    print("For country", country_codes[i], "|", "Delta 2016-2017:", delta, "|", "Value 2016:", value_2016, "|", "Predicted Value 2017:", pred_2017)
    print("---------------------")

For country LUX | Delta 2016-2017: 0.1 | Value 2016: 82.29 | Predicted Value 2017: 82.39
---------------------
For country EST | Delta 2016-2017: -0.06 | Value 2016: 77.74 | Predicted Value 2017: 77.68
---------------------
For country CZE | Delta 2016-2017: -0.1 | Value 2016: 78.33 | Predicted Value 2017: 78.23
---------------------
For country ZMB | Delta 2016-2017: 0.48 | Value 2016: 61.87 | Predicted Value 2017: 62.35
---------------------
For country USA | Delta 2016-2017: 0.09 | Value 2016: 78.69 | Predicted Value 2017: 78.78
---------------------
For country NPL | Delta 2016-2017: 0.36 | Value 2016: 70.25 | Predicted Value 2017: 70.61
---------------------
For country SAU | Delta 2016-2017: 0.17 | Value 2016: 74.56 | Predicted Value 2017: 74.73
---------------------
For country MRT | Delta 2016-2017: 0.16 | Value 2016: 63.24 | Predicted Value 2017: 63.4
---------------------
For country PNG | Delta 2016-2017: 0.18 | Value 2016: 65.54 | Predicted Value 2017: 65.72
---------------

### Truth / Predicted Plot | Life Expectancy

In [442]:
life_expect_2017 = list(pd.read_csv("./data/life_expect2017.csv", header=None)[1])
# plotGraph(preds_life_expect, life_expect_2017, "green", "Life Expectancy")